In [2]:
import os
os.environ['ALFRED_ROOT'] = '/home/hoyeung/alfred/'

In [3]:
import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

In [4]:
# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

In [26]:
import numpy as np
import nn.vnn as vnn
import collections
from torch import nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from model.seq2seq import Module as Base
from models.utils.metric import compute_f1, compute_exact
from nltk.translate.bleu_score import corpus_bleu

In [5]:
# parser
# parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)

class args:
    pass

# settings
# parser.add_argument('--seed', help='random seed', default=123, type=int)
# parser.add_argument('--data', help='dataset folder', default='data/json_feat_2.1.0')
# parser.add_argument('--splits', help='json file containing train/dev/test splits', default='splits/oct21.json')
# parser.add_argument('--preprocess', help='store preprocessed data to json files', action='store_true')
# parser.add_argument('--pp_folder', help='folder name for preprocessed data', default='pp')
# parser.add_argument('--save_every_epoch', help='save model after every epoch (warning: consumes a lot of space)', action='store_true')
# parser.add_argument('--model', help='model to use', default='seq2seq_im')
# parser.add_argument('--gpu', help='use gpu', action='store_true')
# parser.add_argument('--dout', help='where to save model', default='exp/model:{model}')
# parser.add_argument('--resume', help='load a checkpoint')
args.seed = 123
args.data = 'data/json_feat_2.1.0'
args.splits = 'splits/oct21.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.save_every_epoch = False #!
args.model = 'seq2seq_im'
args.gpu = True
args.dout = 'exp/model:seq2seq_im'
args.resume = False #!


# hyper parameters
# parser.add_argument('--batch', help='batch size', default=8, type=int)
# parser.add_argument('--epoch', help='number of epochs', default=20, type=int)
# parser.add_argument('--lr', help='optimizer learning rate', default=1e-4, type=float)
# parser.add_argument('--decay_epoch', help='num epoch to adjust learning rate', default=10, type=int)
# parser.add_argument('--dhid', help='hidden layer size', default=512, type=int)
# parser.add_argument('--dframe', help='image feature vec size', default=2500, type=int)
# parser.add_argument('--demb', help='language embedding size', default=100, type=int)
# parser.add_argument('--pframe', help='image pixel size (assuming square shape eg: 300x300)', default=300, type=int)
# parser.add_argument('--mask_loss_wt', help='weight of mask loss', default=1., type=float)
# parser.add_argument('--action_loss_wt', help='weight of action loss', default=1., type=float)
# parser.add_argument('--subgoal_aux_loss_wt', help='weight of subgoal completion predictor', default=0., type=float)
# parser.add_argument('--pm_aux_loss_wt', help='weight of progress monitor', default=0., type=float)
args.batch = 8
args.epoch = 20
args.lr = 1e-4
args.decay_epoch = 10
args.dhid = 512
args.dframe = 2500
args.demb = 100
args.pframe = 300
args.mask_loss_wt = 1.
args.action_loss_wt = 1.
args.subgoal_aux_loss_wt = 0.
args.pm_aux_loss_wt = 0.



# dropouts
# parser.add_argument('--zero_goal', help='zero out goal language', action='store_true')
# parser.add_argument('--zero_instr', help='zero out step-by-step instr language', action='store_true')
# parser.add_argument('--lang_dropout', help='dropout rate for language (goal + instr)', default=0., type=float)
# parser.add_argument('--input_dropout', help='dropout rate for concatted input feats', default=0., type=float)
# parser.add_argument('--vis_dropout', help='dropout rate for Resnet feats', default=0.3, type=float)
# parser.add_argument('--hstate_dropout', help='dropout rate for LSTM hidden states during unrolling', default=0.3, type=float)
# parser.add_argument('--attn_dropout', help='dropout rate for attention', default=0., type=float)
# parser.add_argument('--actor_dropout', help='dropout rate for actor fc', default=0., type=float)
args.zero_goal = False #!
args.zero_instr = False #!
args.act_dropout = 0.
args.lang_dropout = 0.
args.input_dropout = 0.
args.vis_dropout = 0.3
args.hstate_dropout = 0.3
args.attn_dropout = 0.
args.actor_dropout = 0.


# other settings
# parser.add_argument('--dec_teacher_forcing', help='use gpu', action='store_true')
# parser.add_argument('--temp_no_history', help='use gpu', action='store_true')
args.dec_teacher_forcing = False #!
args.temp_no_history = False #!


# debugging
# parser.add_argument('--fast_epoch', help='fast epoch during debugging', action='store_true')
# parser.add_argument('--dataset_fraction', help='use fraction of the dataset for debugging (0 indicates full size)', default=0, type=int)
args.fast_epoch = False #!
args.dataset_fraction = 0

In [6]:
args.splits = 'data/splits/oct21.json'
args.dec_teacher_forcing = True

In [7]:
torch.manual_seed(args.seed)

In [21]:
# make output dir
if not os.path.isdir(args.dout):
    os.makedirs(args.dout)

print('args.out ', args.dout)

args.out  exp/model:seq2seq_im


In [9]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 21023,
 'valid_seen': 820,
 'valid_unseen': 821}


In [22]:
# preprocess and save
if args.preprocess:
    print("\nPreprocessing dataset and saving to %s folders ... This will take a while. Do this once as required." % args.pp_folder)
    dataset = Dataset(args, None)
    dataset.preprocess_splits(splits)
    vocab = torch.load(os.path.join(args.dout, "%s.vocab" % args.pp_folder))
else:
    vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))
    
print(vocab)

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}


In [ ]:
class Seq2SeqNL(Base):

    def __init__(self, args, vocab):
        '''
        Seq2Seq agent
        '''
        super(NLSeq2Seq, self).__init__(args, vocab)

        # encoder and self-attention
        self.enc = nn.LSTM(args.demb, args.dhid, bidirectional=True, batch_first=True)
        self.enc_att = vnn.SelfAttn(args.dhid*2)

#         # frame mask decoder
#         decoder = vnn.ConvFrameMaskDecoderProgressMonitor if self.subgoal_monitoring else vnn.ConvFrameMaskDecoder
#         self.dec = decoder(self.emb_action_low, args.dframe, 2*args.dhid,
#                            pframe=args.pframe,
#                            attn_dropout=args.attn_dropout,
#                            hstate_dropout=args.hstate_dropout,
#                            actor_dropout=args.actor_dropout,
#                            input_dropout=args.input_dropout,
#                            teacher_forcing=args.dec_teacher_forcing)

        self.dec = ???

        # dropouts
#         self.lang_dropout = nn.Dropout(args.lang_dropout, inplace=True)
        self.input_dropout = nn.Dropout(args.input_dropout)

        # internal states
        self.state_t = None
        self.e_t = None
        self.test_mode = False

        # bce reconstruction loss
        self.lang_loss = ???
#         self.mse_loss = torch.nn.MSELoss(reduction='none')

        # paths
        self.root_path = os.getcwd()

        # params
        self.max_subgoals = 25

        # reset model
        self.reset()

#     def iterate(self, data, batch_size):
#         '''
#         breaks dataset into batch_size chunks for training
#         '''
#         for i in trange(0, len(data), batch_size, desc='batch'):
#             tasks = data[i:i+batch_size]
#             batch = [self.load_task_json(task) for task in tasks]
#             feat = self.featurize(batch)
#             yield batch, feat
        
    def featurize(batch):
        '''tensoroze and pad batch input'''
    
        device = torch.device('cuda') if self.args.gpu else torch.device('cpu')
        feat = collections.defaultdict(list)
    
        for ex in batch:
            
            #########
            # outputs
            #########
            
            # serialize segments
            self.serialize_lang_action(ex)
            
            # goal and instr language
            lang_goal, lang_instr = ex['num']['lang_goal'], ex['num']['lang_instr']
            
            # zero inputs if specified
            lang_goal = self.zero_input(lang_goal) if self.args.zero_goal else lang_goal
            lang_instr = self.zero_input(lang_instr) if self.args.zero_instr else lang_instr
            
            # append goal
            feat['lang_goal'].append(lang_goal)
            
            # append instr
            feat['lang_instr'].append(lang_instr)
            
            # append goal + instr
            lang_goal_instr = lang_goal + lang_instr
            feat['lang_goal_instr'].append(lang_goal_instr)
            
            # Skipped loading Resnet features from disk
            
            #########
            # inputs
            #########
            
            if not self.test_mode:
                # low-level action
                feat['action_low'].append([a['action'] for a in ex['num']['action_low']])
        
        # TODO -- embed action in addition to language
        # tensorization and padding                
        for k, v in feat.items():
            if k in {'lang_goal_instr', 'lang_goal', 'lang_instr'}:
                # language embedding and padding
                seqs = [torch.tensor(vv, device=device) for vv in v]
                pad_seq = pad_sequence(seqs, batch_first=True, padding_value=self.pad)
                seq_lengths = np.array(list(map(len, v)))
                embed_seq = self.emb_word(pad_seq)
                packed_input = pack_padded_sequence(embed_seq, seq_lengths, batch_first=True, enforce_sorted=False)
                feat[k] = packed_input
            else:
                # default: tensorize and pad sequence
                seqs = [torch.tensor(vv, device=device, dtype=torch.float if ('frames' in k) else torch.long) for vv in v]
                pad_seq = pad_sequence(seqs, batch_first=True, padding_value=self.pad)
                feat[k] = pad_seq             
    
        return feat
    
    def serialize_lang_action(self, feat):
        '''
        append segmented instr language and low-level actions into single sequences
        '''
        is_serialized = not isinstance(feat['num']['lang_instr'][0], list)
        if not is_serialized:
            feat['num']['lang_instr'] = [word for desc in feat['num']['lang_instr'] for word in desc]
            if not self.test_mode:
                feat['num']['action_low'] = [a for a_group in feat['num']['action_low'] for a in a_group]

    def forward(self, feat, max_decode=300):
        cont_act, enc_act = self.encode_act(feat)
        state_0 = cont_act, torch.zeros.like(cont_act)
        res = self.dec(enc_act, max_decode=300, gold=feat['???'], state_0=state_0)
        feat.update(res)
        return feat
    
    def encode_act(self, feat):
        '''
        encode low-level actions
        '''    
        pass
    
    def reset(self):
        pass
    
    def step(self, feat, prev_action=None):
        '''
        forward the model for a single time-step (used for real-time execution during eval)
        '''
        # DO WE NEED THIS?
        pass
    
    def extract_preds(self, out, batch, feat, clean_special_tokens=True):
        # MAY NOT NEED THIS
        return preds

    def embed_action(self, action):
        '''
        embed low-level action
        '''
        pass
    
#     def weighted_mask_loss(self, pred_masks, gt_masks):
#     def flip_tensor(self, tensor, on_zero=1, on_non_zero=0):
    
    def compute_loss(self, out, batch, feat):
        pass
    
    def compute_metric(self, preds, data):
        pass
    
    
    